In [ ]:
import os, time, random, logging , datetime , cv2 , csv , subprocess
import numpy as np


import tensorflow as tf
print("tf",tf.version.VERSION)
from tensorflow import keras

from utils import globo ,  xdv , tfh5


''' GPU CONFIGURATION '''

#os.environ["CUDA_VISIBLE_DEVICES"]="1,2"
#tfh5.set_tf_loglevel(logging.ERROR)
#tfh5.tf.debugging.set_log_device_placement(False) #Enabling device placement logging causes any Tensor allocations or operations to be printed.
#tfh5.set_memory_growth()
#os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


In [1]:
import essentia.standard as es
print(dir(es))

2023-04-30 22:37:06.553401: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-30 22:37:06.553449: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


: 

: 

In [ ]:
''' TRAIN & VALDT '''
#train_fn, train_labels, train_tot_frames, valdt_fn, valdt_labels , valdt_tot_frames = xdv.train_valdt_files(tframes=True)
train_fn, train_labels, valdt_fn, valdt_labels = xdv.train_valdt_files()

In [ ]:
''' CONFIGS '''

train_config_wav = {
    "frame_step":2, #24 fps -> 12
    
    "sinet_aas_len" : 8,
    "batch_size" : 1,
    
    "augment" : True,
    "shuffle" : False,
    
    "ativa" : 'relu',
    "optima" : 'sgd',
    "batch_type" : 0,   # =0 all batch have frame_max or video length // =1 last batch has frame_max frames // =2 last batch has no repetead frames
    "frame_max" : 8000,
    "ckpt_start" : f"{0:0>8}",  #used in train_model: if 00000000 start from scratch, else start from ckpt with config stated
    
    "epochs" : 1
}
